# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 10:42AM,257935,10,0086419772,ISDIN Corporation,Released
1,Feb 22 2023 10:42AM,257935,10,0086420055,ISDIN Corporation,Released
2,Feb 22 2023 10:42AM,257935,10,0086420126,ISDIN Corporation,Released
3,Feb 22 2023 10:42AM,257935,10,0086420128,ISDIN Corporation,Released
4,Feb 22 2023 10:42AM,257935,10,0086420130,ISDIN Corporation,Released
5,Feb 22 2023 10:42AM,257935,10,0086420064,ISDIN Corporation,Released
6,Feb 22 2023 10:42AM,257935,10,0086420068,ISDIN Corporation,Released
7,Feb 22 2023 10:42AM,257935,10,0086420066,ISDIN Corporation,Released
8,Feb 22 2023 10:42AM,257935,10,0086420065,ISDIN Corporation,Released
9,Feb 22 2023 10:42AM,257935,10,0086420081,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
46,257930,Released,1
47,257931,Released,1
48,257933,Executing,1
49,257934,Released,2
50,257935,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257930,NaN,NaN,1.0
257931,NaN,NaN,1.0
257933,NaN,1.0,NaN
257934,NaN,NaN,2.0
257935,NaN,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257832,4.0,0.0,4.0
257837,1.0,0.0,2.0
257845,0.0,0.0,2.0
257852,0.0,0.0,1.0
257858,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257832,4,0,4
257837,1,0,2
257845,0,0,2
257852,0,0,1
257858,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257832,4,0,4
1,257837,1,0,2
2,257845,0,0,2
3,257852,0,0,1
4,257858,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257832,4,,4
1,257837,1,,2
2,257845,,,2
3,257852,,,1
4,257858,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation
17,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation
19,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc."
20,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions
21,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions
22,Feb 22 2023 10:07AM,257929,20,"HVL, LLC dba Atrium Innovations"
23,Feb 22 2023 10:02AM,257928,10,"ClearSpec, LLC"
24,Feb 22 2023 10:00AM,257927,19,"Methapharm, Inc."
26,Feb 22 2023 9:58AM,257926,18,"Emersa Waterbox, LLC"
27,Feb 22 2023 9:56AM,257925,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation,,,17
1,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation,,,2
2,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc.",,1,
3,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions,,,1
4,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions,,,1
5,Feb 22 2023 10:07AM,257929,20,"HVL, LLC dba Atrium Innovations",,,1
6,Feb 22 2023 10:02AM,257928,10,"ClearSpec, LLC",,,1
7,Feb 22 2023 10:00AM,257927,19,"Methapharm, Inc.",,,2
8,Feb 22 2023 9:58AM,257926,18,"Emersa Waterbox, LLC",,,1
9,Feb 22 2023 9:56AM,257925,19,VITABIOTICS USA INC.,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation,17,,
1,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation,2,,
2,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc.",,1,
3,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions,1,,
4,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions,1,,
5,Feb 22 2023 10:07AM,257929,20,"HVL, LLC dba Atrium Innovations",1,,
6,Feb 22 2023 10:02AM,257928,10,"ClearSpec, LLC",1,,
7,Feb 22 2023 10:00AM,257927,19,"Methapharm, Inc.",2,,
8,Feb 22 2023 9:58AM,257926,18,"Emersa Waterbox, LLC",1,,
9,Feb 22 2023 9:56AM,257925,19,VITABIOTICS USA INC.,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation,17,,
1,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation,2,,
2,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc.",,1,
3,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions,1,,
4,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation,17.0,NaN,NaN
1,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation,2.0,NaN,NaN
2,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc.",NaN,1.0,NaN
3,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 10:42AM,257935,10,ISDIN Corporation,17.0,0.0,0.0
1,Feb 22 2023 10:21AM,257934,10,ISDIN Corporation,2.0,0.0,0.0
2,Feb 22 2023 10:16AM,257933,21,"NBTY Global, Inc.",0.0,1.0,0.0
3,Feb 22 2023 10:09AM,257931,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Feb 22 2023 10:07AM,257930,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6963009,182.0,3.0,17.0
12,773706,6.0,2.0,0.0
15,257890,69.0,0.0,0.0
16,515861,2.0,0.0,0.0
18,515846,2.0,0.0,0.0
19,1805285,10.0,28.0,2.0
20,257929,1.0,0.0,0.0
21,257933,0.0,1.0,0.0
22,773663,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6963009,182.0,3.0,17.0
1,12,773706,6.0,2.0,0.0
2,15,257890,69.0,0.0,0.0
3,16,515861,2.0,0.0,0.0
4,18,515846,2.0,0.0,0.0
5,19,1805285,10.0,28.0,2.0
6,20,257929,1.0,0.0,0.0
7,21,257933,0.0,1.0,0.0
8,22,773663,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,182.0,3.0,17.0
1,12,6.0,2.0,0.0
2,15,69.0,0.0,0.0
3,16,2.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,10.0,28.0,2.0
6,20,1.0,0.0,0.0
7,21,0.0,1.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,182.0
1,12,Released,6.0
2,15,Released,69.0
3,16,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,17.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Executing,3.0,2.0,0.0,0.0,0.0,28.0,0.0,1.0,0.0
Released,182.0,6.0,69.0,2.0,2.0,10.0,1.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,17.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,3.0,2.0,0.0,0.0,0.0,28.0,0.0,1.0,0.0
2,Released,182.0,6.0,69.0,2.0,2.0,10.0,1.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,17.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,3.0,2.0,0.0,0.0,0.0,28.0,0.0,1.0,0.0
2,Released,182.0,6.0,69.0,2.0,2.0,10.0,1.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()